In [ ]:
# imports
import os
import pandas as pd  # 2.1.3
from matplotlib import pyplot as plt  # 3.8.2
import seaborn as sns  # 0.13.0
import numpy as np  # 1.26.2

from config import __DEBUG__, ___SAVE_IMAGES__, worker_output_dict, shipping_value_dict
import values

In [ ]:
def save_plot_as_file(plot, filename):

    directory = "images"
    file_format = "png"

    if not os.path.exists(directory):
        os.makedirs(directory)

    file_path = os.path.join(directory, f"{filename}.{file_format}")
    plot.savefig(file_path, format=file_format)

In [ ]:
# Setup
# EDIT THIS SECTION ACCORDING TO YOUR GAMEPLAY

maps_per_hour = 10
hours_per_week = 20


# Only edit this if you want to use non-calculated custom values
# Note that this is the average per total maps, not if found
# e.g. average iron is 1000 but only found every 5th map => 200
# TODO: (low prio) Refactor this in way that it allows to enter the average per map if found and then the number of maps it is found in
values_per_map = {
    values.GOLD: 0,
    values.IRON: 0,
    values.ORICHALCUM: 0,
    values.AMBER: 0,
    values.BISMUTH: 0,
    values.VERISIUM: 0,
}

In [ ]:
# load ore data
ore_data = pd.read_csv("settler_ore.csv", sep=";")

if __DEBUG__:
    print(ore_data.head())
    print(len(ore_data))

In [ ]:
# replace NaN in 'influence' and 'comment' with empty string
ore_data["comment"].fillna("", inplace=True)
ore_data["influence"].fillna("", inplace=True)

# replace all NaN with 0
ore_data.fillna(0, inplace=True)

# drop duplicates
ore_data.drop_duplicates(inplace=True)

if __DEBUG__:
    print(ore_data.head())
    print(len(ore_data))

In [ ]:
# remove outliers
# TODO: remove outliers

In [ ]:
# print averages for each ore
avg_per_map = {}

for ore in values.ORES_GOLD:
    print(f"Average for {ore}:")
    non_zero_values = ore_data[ore].loc[ore_data[ore] != 0]
    mean_value = non_zero_values.mean()
    print(f"{mean_value:.2f}")
    avg_per_map[ore] = mean_value

In [ ]:
# print how often each ore is found
ore_count_per_map = {}

total_maps = len(ore_data)
for ore in values.ORES_GOLD:
    ore_count = len(ore_data[ore].loc[ore_data[ore] != 0])
    print(
        f"{ore} found every {total_maps / ore_count:.2f} ({100*ore_count/total_maps:.2f}%) maps"
    )
    print(f"{ore_count} of {total_maps} maps\n")
    ore_count_per_map[ore] = ore_count / total_maps

filtered_ore_data = ore_data.loc[
    ore_data[values.ORES].apply(lambda row: (row != 0).sum() >= 2, axis=1)
]
print(f"{len(filtered_ore_data)} maps with at least 2 ores")

In [ ]:
# calculate values per map if not set
for ore in values.ORES_GOLD:
    if not values_per_map[ore]:
        values_per_map[ore] = avg_per_map[ore] * ore_count_per_map[ore]
    print(f"average {ore} per map: {values_per_map[ore]:.2f}")

In [ ]:
maps_per_week = maps_per_hour * hours_per_week

if __DEBUG__:
    print(f"maps per week: {maps_per_week}")

In [ ]:
# calculate values per week
values_per_week = {}

for ore in values.ORES_GOLD:
    values_per_week[ore] = values_per_map[ore] * maps_per_week
    print(f"{values_per_week[ore]:.0f} {ore} per week")

In [ ]:
# plot the output per ore and per level for mining

mining_dict = worker_output_dict["mining"]

mining_output = pd.DataFrame(mining_dict)

plt.figure(figsize=(10, 6))
for ore in values.ORES:
    sns.lineplot(
        data=mining_output, x=mining_output.index, y=ore, label=ore, marker="o"
    )
    for i in range(len(mining_output)):
        plt.text(
            mining_output.index[i],
            mining_output[ore].iloc[i],
            f"{mining_output[ore].iloc[i]}",
            fontsize=9,
            ha="right",
        )

plt.title("Mining: Output per rank and per hour")
plt.xlabel("Rank")
plt.ylabel("Ore per hour")
plt.legend(title="Ore")
plt.grid(True)
plt.tight_layout()
plt.show()

if ___SAVE_IMAGES__:
    save_plot_as_file(plt, "mining_output_per_rank_per_hour")

save_plot_as_file(plt, "mining_output_per_rank_per_hour")

In [ ]:
# plot the output per ore and per level for smelting

smelting_dict = worker_output_dict["smelting"]

smelting_output = pd.DataFrame(smelting_dict)

smelting_output = smelting_output.dropna()

plt.figure(figsize=(10, 6))
for ore in values.ORES:
    sns.lineplot(
        data=smelting_output, x=smelting_output.index, y=ore, label=ore, marker="o"
    )
    for i in range(len(smelting_output)):
        plt.text(
            smelting_output.index[i],
            smelting_output[ore].iloc[i],
            f"{smelting_output[ore].iloc[i]}",
            fontsize=9,
            ha="right",
        )

plt.title("Smelting: Output per rank and per hour")
plt.xlabel("Rank")
plt.ylabel("Ore per hour")
plt.legend(title="Ore")
plt.grid(True)
plt.tight_layout()
plt.show()

if ___SAVE_IMAGES__:
    save_plot_as_file(plt, "smelting_output_per_rank_per_hour")

In [ ]:
# plot the output per crop and per level for farming

farming_dict = worker_output_dict["farming"]

farming_output = pd.DataFrame(farming_dict)

plt.figure(figsize=(10, 6))
for crop in values.CROPS:
    sns.lineplot(
        data=farming_output, x=farming_output.index, y=crop, label=crop, marker="o"
    )
    for i in range(len(farming_output)):
        plt.text(
            farming_output.index[i],
            farming_output[crop].iloc[i],
            f"{farming_output[crop].iloc[i]}",
            fontsize=9,
            ha="right",
        )

plt.title("Farming: Output per rank and per hour")
plt.xlabel("Rank")
plt.ylabel("Crop per hour")
plt.legend(title="Crop")
plt.grid(True)
plt.tight_layout()
plt.show()

if ___SAVE_IMAGES__:
    save_plot_as_file(plt, "farming_output_per_rank_per_hour")

In [ ]:
# plot the value output per crop and per level for farming

farming_dict = worker_output_dict["farming"]

farming_output = pd.DataFrame(farming_dict)

for crop in values.CROPS:
    farming_output[crop] = farming_output[crop] * shipping_value_dict[crop]

plt.figure(figsize=(10, 6))
for crop in values.CROPS:
    sns.lineplot(
        data=farming_output, x=farming_output.index, y=crop, label=crop, marker="o"
    )
    for i in range(len(farming_output)):
        plt.text(
            farming_output.index[i],
            farming_output[crop].iloc[i],
            f"{farming_output[crop].iloc[i]}",
            fontsize=9,
            ha="right",
        )

plt.title("Output per rank and per hour")
plt.xlabel("Rank")
plt.ylabel("Crop per hour")
plt.legend(title="Crop")
plt.grid(True)
plt.tight_layout()
plt.show()

display(farming_output)

In [ ]:
# load worker data
worker_data = pd.read_csv("settler_worker.csv", sep=";")

if __DEBUG__:
    print(worker_data.head())
    print(len(worker_data))

In [ ]:
# remove NaN
worker_data.fillna(0, inplace=True)

# drop duplicates
# worker_data.drop_duplicates(inplace=True)

if __DEBUG__:
    print(worker_data.head())
    print(len(worker_data))

In [ ]:
# box plot the wage
worker_data.boxplot(column="wage", rot=90)

In [ ]:
# display a table with the number of occurrences of each rank for each skill

# Create an empty dictionary to store the counts
skill_counts = {}
# Populate the dictionary with the value counts for each skill
for skill in values.SKILLS:
    # only include worker data with only one skill
    skill_counts[skill] = worker_data.loc[
        (worker_data[skill] != 0)
        & worker_data[values.SKILLS].apply(lambda row: (row != 0).sum() == 1, axis=1)
    ][skill].value_counts()

# Convert the dictionary to a DataFrame
skill_counts_df = pd.DataFrame(skill_counts).fillna(0).astype(int)


skill_counts_df.index = skill_counts_df.index.astype(int)
skill_counts_df["Rank"] = skill_counts_df.index
skill_counts_df.set_index("Rank", inplace=True)

# Create a custom color map
cmap = sns.color_palette("YlGnBu", as_cmap=True)
cmap.set_under("red")

# Plot the heatmap for all skills
plt.figure(figsize=(12, 8))
sns.heatmap(skill_counts_df, annot=True, fmt="d", cmap=cmap, cbar=True, vmin=0.1)
plt.title("Heatmap of Skill Counts by Rank (All Skills)")
plt.xlabel("Skills")
plt.ylabel("Rank")
plt.show()

If there are red tiles present, the dataset is likely to small because there is not even a datapoint for every rank for each skill.

In [ ]:
# plot the average wage grouped by skills

for skill in values.SKILLS:
    ax = plt.subplot()
    subset_data = worker_data.loc[(worker_data[skill] != 0)]
    subset_data.groupby(skill)["wage"].mean().plot(kind="line", ax=ax, label="all")
    for i in range(1, len(values.SKILLS)):
        subset_data.loc[
            subset_data[values.SKILLS].apply(lambda row: (row != 0).sum() == i, axis=1)
        ].groupby(skill)["wage"].mean().plot(kind="line", ax=ax, label=f"{i} skills")

    plt.legend()
    plt.xticks(range(1, 11))
    plt.xlabel("rank")
    plt.ylabel("wage per hour in gold")
    plt.title(f"Average wage grouped by {skill}")
    plt.show()

In [ ]:
# plot the average wage grouped by skills

marker_dict = {
    values.MINING: "o",
    values.SMELTING: "s",
    values.FARMING: "D",
    values.MAPPING: "P",
    values.SHIPPING: "X",
    values.DISENTCHANTING: "v",
}

for skill in values.SKILLS:
    ax = plt.subplot()
    worker_data.loc[(worker_data[skill] != 0)].groupby(skill)["wage"].mean().plot(
        kind="line", ax=ax, label="all"
    )
    single_skill = worker_data.loc[
        worker_data[values.SKILLS].apply(lambda row: (row != 0).sum() == 1, axis=1)
        & (worker_data[skill] != 0)
    ]
    single_skill.groupby(skill)["wage"].mean().plot(kind="line", ax=ax, label="single")
    worker_data.loc[
        worker_data[values.SKILLS].apply(lambda row: (row != 0).sum() <= 2, axis=1)
        & (worker_data[skill] != 0)
    ].groupby(skill)["wage"].mean().plot(kind="line", ax=ax, label="duo")
    scatter = ax.scatter(
        single_skill[skill],
        single_skill["wage"],
        label=skill,
        c=single_skill[skill],
        marker=marker_dict[skill],
    )
    for sub_skill in values.SKILLS:
        if sub_skill == skill:
            continue
        other_skill = worker_data.loc[
            worker_data[values.SKILLS].apply(lambda row: (row != 0).sum() == 2, axis=1)
            & (worker_data[sub_skill] != 0)
            & (worker_data[skill] != 0)
        ]
        ax.scatter(
            other_skill[skill],
            other_skill["wage"],
            label=sub_skill,
            c=other_skill[sub_skill],
            marker=marker_dict[sub_skill],
        )

    plt.legend()
    plt.xticks(range(1, 11))
    plt.xlabel("rank")
    plt.ylabel("wage per hour in gold")
    plt.title(f"Average wage grouped by {skill}")
    plt.colorbar(scatter, label="Rank")
    plt.show()

In [ ]:
ax = plt.subplot()

polynomial_formulas = {}

for skill in values.SKILLS:
    skill_data = (
        worker_data.loc[
            worker_data[values.SKILLS].apply(lambda row: (row != 0).sum() == 1, axis=1)
            & (worker_data[skill] != 0)
        ]
        .groupby(skill)["wage"]
        .mean()
    )

    skill_data.plot(kind="line", ax=ax, label=skill, marker="o")

    # Fit a polynomial of degree 2 (quadratic) to the data
    x = skill_data.index
    y = skill_data.values
    coeffs = np.polyfit(x, y, 2)
    poly = np.poly1d(coeffs)

    # Store the polynomial formula in the dictionary
    polynomial_formulas[skill] = poly

    # Plot the fitted polynomial curve
    x_fit = np.linspace(x.min(), 10, 100)
    y_fit = poly(x_fit)
    plt.plot(x_fit, y_fit, linestyle="--", label=f"{skill} fit")

plt.xticks(range(1, 11))
plt.legend()
plt.xlabel("Rank")
plt.ylabel("Wage per hour in gold")
plt.title("Average Wage with Polynomial Fit")
plt.grid(True)
plt.show()

# Print the polynomial formulas
for skill, poly in polynomial_formulas.items():
    print(f"{skill}: {poly}")

if ___SAVE_IMAGES__:
    save_plot_as_file(plt, "average_wage_per_rank")

This is quite useful information, because it can be seen that there seems to be a clear ordering in skills regarding the wages, which means that a person even with low level mapping  and mid level mining has likely a higher wage than a person with only mid level mining.
So getting good data on the wages of mappers is the easiest, because it is possible to include more data. The price of other skills may be inflated due to a secondary talent.

Based on the polynom fitting a wage per skill is calculated and the hightest one is used as a threshold with an offset to avoid removing too many skills, especially in the lower ranks.
As example: A worker with rank 10 mapping and rank 1 mining will have only rank 10 mapping afterwards.

After these trimming it is now possible to successfully include workers which have multiple ranks. It would be possible to do this by hand and manually ranking each skill, but in this way it is not necessary to find rules for the trimming, like rank 5 mapper has higher wage than rank 8 smelting.

In [ ]:
# We define an offset to include values in the lower ranks too
# While this has a negative impact on the data quality, is is necessary to include lower ranks
offset = 25

# use the fitting to predict the wage for each rank
for skill in values.SKILLS:
    worker_data[f"{skill}_wage"] = worker_data[skill].apply(
        lambda rank: polynomial_formulas[skill](rank) if rank != 0 else 0
    )

if __DEBUG__:
    print(worker_data.head())

# calculate the threshold wage
worker_data["threshold_wage"] = (
    worker_data[[f"{skill}_wage" for skill in values.SKILLS]].max(axis=1)
) - offset

if __DEBUG__:
    print(worker_data.head())

# set other skill columns to zero which are below the threshold wage
for skill in values.SKILLS:
    worker_data[skill] = worker_data.apply(
        lambda row: row[skill] if row[f"{skill}_wage"] >= row["threshold_wage"] else 0,
        axis=1,
    )

if __DEBUG__:
    print(worker_data.head())

# drop created columns
worker_data.drop(
    columns=[f"{skill}_wage" for skill in values.SKILLS] + ["threshold_wage"],
    inplace=True,
)

if __DEBUG__:
    print(worker_data.head())

In [ ]:
# display a table with the number of occurrences of each rank for each skill

# Create an empty dictionary to store the counts
skill_counts = {}
# Populate the dictionary with the value counts for each skill
for skill in values.SKILLS:
    skill_counts[skill] = worker_data.loc[
        (worker_data[skill] != 0)
        & worker_data[values.SKILLS].apply(lambda row: (row != 0).sum() == 1, axis=1)
    ][skill].value_counts()

# Convert the dictionary to a DataFrame
skill_counts_df = pd.DataFrame(skill_counts).fillna(0).astype(int)


skill_counts_df.index = skill_counts_df.index.astype(int)
skill_counts_df["Rank"] = skill_counts_df.index
skill_counts_df.set_index("Rank", inplace=True)

# Create a custom color map
cmap = sns.color_palette("YlGnBu", as_cmap=True)
cmap.set_under("red")

# Plot the heatmap for all skills
plt.figure(figsize=(12, 8))
sns.heatmap(skill_counts_df, annot=True, fmt="d", cmap=cmap, cbar=True, vmin=0.1)
plt.title("Heatmap of Skill Counts by Rank (All Skills)")
plt.xlabel("Skills")
plt.ylabel("Rank")
plt.show()

In comparision to the previous heatmap the data could be improved to include more ranks.

In [ ]:
# plot average wage for each skill per rank

ax = plt.subplot()

polynomial_formulas_refined = {}

for skill in values.SKILLS:
    skill_data = (
        worker_data.loc[
            worker_data[values.SKILLS].apply(lambda row: (row != 0).sum() == 1, axis=1)
            & (worker_data[skill] != 0)
        ]
        .groupby(skill)["wage"]
        .mean()
    )

    skill_data.plot(kind="line", ax=ax, label=skill, marker="o")

    # Fit a polynomial of degree 2 (quadratic) to the data
    x = skill_data.index
    y = skill_data.values
    coeffs = np.polyfit(x, y, 2)
    poly = np.poly1d(coeffs)

    # Store the polynomial formula in the dictionary
    polynomial_formulas_refined[skill] = poly

    # Plot the fitted polynomial curve
    x_fit = np.linspace(x.min(), 10, 100)
    y_fit = poly(x_fit)
    plt.plot(x_fit, y_fit, linestyle="--", label=f"{skill} fit")

plt.xticks(range(1, 11))
plt.legend()
plt.xlabel("Rank")
plt.ylabel("Wage per hour in gold")
plt.title("Average Wage with Polynomial Fit")
plt.grid(True)
plt.show()

# Create a DataFrame from the polynomial formulas
poly_df = pd.DataFrame(
    {
        "Skill": polynomial_formulas_refined.keys(),
        "Polynomial": [poly for poly in polynomial_formulas_refined.values()],
    }
)

# convert the polynomial to a string
poly_df["Polynomial"] = poly_df["Polynomial"].apply(
    lambda poly: " + ".join(
        [
            (
                f"{coeff:.4f} * x^{len(poly.c) - i - 1}"
                if (len(poly.c) - i - 1) > 0
                else f"{coeff:.4f}"
            )
            for i, coeff in enumerate(poly.c)
        ]
    )
)

# Display the DataFrame as a table
display(poly_df)

if ___SAVE_IMAGES__:
    save_plot_as_file(plt, "average_wage_per_rank_cleaned")

This dataset is still too small to include wage averages for every rank and skill, therefore the refined fitting is used from now on.
When the dataset quality improves enough to include 

In [ ]:
# calcuate the wage for each rank and skill

wage_dict = {}

for skill in values.SKILLS:
    wage_dict[skill] = {
        rank: polynomial_formulas_refined[skill](rank) for rank in range(1, 11)
    }

wage_df = pd.DataFrame(wage_dict)
wage_df = wage_df.map(lambda x: f"{x:.2f}")

display(wage_df)

In [ ]:
# Calculate the percentage for each average of the total per rank

wage_percentage_dict = {}

for skill in values.SKILLS:
    wage_percentage_dict[skill] = {
        rank: polynomial_formulas_refined[skill](rank) for rank in range(1, 11)
    }

wage_percentage_df = pd.DataFrame(wage_percentage_dict)

# Calculate the total sum for each rank and append it as a new column
wage_percentage_df["Total"] = wage_percentage_df.sum(axis=1)

# Calculate the percentage of the total for each skill per rank and add as new columns
for skill in values.SKILLS:
    wage_percentage_df[f"{skill} %"] = (
        (wage_percentage_df[skill] / wage_percentage_df["Total"]) * 100
    ).round(2)
    wage_percentage_df[skill] = wage_percentage_df[skill].map(lambda x: f"{x:.2f}")

# Order the columns so that the percentage column is next to each skill column
ordered_columns = []
for skill in values.SKILLS:
    ordered_columns.append(skill)
    ordered_columns.append(f"{skill} %")
ordered_columns.append("Total")

wage_percentage_df = wage_percentage_df[ordered_columns]


display(wage_percentage_df)

In [ ]:
# Mining: calculate the wage for 100 mined ores
mining_wage = wage_dict[values.MINING]
mining_output = worker_output_dict[values.MINING]

# Dictionary to store the cost per 100 units for each ore
mining_cost_per_100_units = {ore: [] for ore in values.ORES}

for ore in values.ORES:
    print(f"Mining {ore}:")
    for rank in range(1, 11):
        wage = mining_wage[rank]
        output = mining_output[ore][rank]
        if not output:
            mining_cost_per_100_units[ore].append(None)
            continue
        cost = wage / output * 100
        mining_cost_per_100_units[ore].append(cost)
        if __DEBUG__:
            print(f"Rank {rank}: {cost:.2f} gold per 100 {ore}")
    print()

# Create a DataFrame from the mining costs
mining_df = pd.DataFrame(mining_cost_per_100_units, index=range(1, 11))

# Plot the cost per 100 units for each ore
fig, ax = plt.subplots()

for ore, costs in mining_cost_per_100_units.items():
    ranks = range(1, 11)
    ax.plot(ranks, costs, label=ore, marker="o")

ax.set_xticks(range(1, 11))
ax.set_xlabel("Rank")
ax.set_ylabel("Cost per 100 units (gold)")
ax.set_title("Cost per 100 units of mined ores")
ax.legend()
ax.grid(True)

plt.show()

if ___SAVE_IMAGES__:
    save_plot_as_file(plt, "mining_cost_per_100_units")

In [ ]:
# Smelting: calculate the wage for 100 mined ores
smelting_wage = wage_dict[values.SMELTING]
smelting_output = worker_output_dict[values.SMELTING]

# Dictionary to store the cost per 100 units for each ore
smelting_cost_per_100_units = {ore: [] for ore in values.ORES}

for ore in values.ORES:
    print(f"Smelting {ore}:")
    for rank in range(1, 11):
        wage = smelting_wage[rank]
        output = smelting_output[ore][rank]
        if not output:
            smelting_cost_per_100_units[ore].append(None)
            continue
        cost = wage / output * 100
        smelting_cost_per_100_units[ore].append(cost)
        if __DEBUG__:
            print(f"Rank {rank}: {cost:.2f} gold per 100 {ore}")
    print()

# Create a DataFrame from the smelting costs
smelting_df = pd.DataFrame(smelting_cost_per_100_units, index=range(1, 11))

# Plot the cost per 100 units for each ore
fig, ax = plt.subplots()

for ore, costs in smelting_cost_per_100_units.items():
    ranks = range(1, 11)
    ax.plot(ranks, costs, label=ore, marker="o")

ax.set_xticks(range(1, 11))
ax.set_xlabel("Rank")
ax.set_ylabel("Cost per 100 units (gold)")
ax.set_title("Cost per 100 units of smelted ores")
ax.legend()
ax.grid(True)

plt.show()

if ___SAVE_IMAGES__:
    save_plot_as_file(plt, "smelting_cost_per_100_units")

For mining the higher and lower ranks are more efficient than the middle ranks, while for smelting the lower ranks are more efficient, but there is only a small difference between the middle and high ranks.

In [ ]:
# get min and max cost per 100 units for each ore for mining and smelting and plot them

# Find the min and max values along with their ranks for each ore (Mining)
mining_min_max_stats = []

for ore in mining_df.columns:
    mining_min_value = mining_df[ore].min()
    mining_max_value = mining_df[ore].max()
    mining_min_rank = mining_df[ore].idxmin() + 1
    mining_max_rank = mining_df[ore].idxmax() + 1
    mining_min_max_stats.append(
        {
            "Ore": ore,
            "Mining Min Cost": f"{mining_min_value:.2f}",
            "Mining Min Rank": mining_min_rank,
            "Mining Max Cost": f"{mining_max_value:.2f}",
            "Mining Max Rank": mining_max_rank,
        }
    )

# Convert the list of dictionaries to a DataFrame
mining_min_max_df = pd.DataFrame(mining_min_max_stats)

# Find the min and max values along with their ranks for each ore (Smelting)
smelting_min_max_stats = []

for ore in smelting_df.columns:
    smelting_min_value = smelting_df[ore].min()
    smelting_max_value = smelting_df[ore].max()
    smelting_min_rank = smelting_df[ore].idxmin() + 1
    smelting_max_rank = smelting_df[ore].idxmax() + 1
    smelting_min_max_stats.append(
        {
            "Ore": ore,
            "Smelting Min Cost": f"{smelting_min_value:.2f}",
            "Smelting Min Rank": smelting_min_rank,
            "Smelting Max Cost": f"{smelting_max_value:.2f}",
            "Smelting Max Rank": smelting_max_rank,
        }
    )

# Convert the list of dictionaries to a DataFrame
smelting_min_max_df = pd.DataFrame(smelting_min_max_stats)

# combine both dataframes
mining_smelting_min_max_df = pd.merge(mining_min_max_df, smelting_min_max_df, on="Ore")

# add columns for addition of min and max for mining and smelting
mining_smelting_min_max_df["Min Total"] = mining_smelting_min_max_df[
    "Mining Min Cost"
].astype(float) + mining_smelting_min_max_df["Smelting Min Cost"].astype(float)
mining_smelting_min_max_df["Max Total"] = mining_smelting_min_max_df[
    "Mining Max Cost"
].astype(float) + mining_smelting_min_max_df["Smelting Max Cost"].astype(float)
mining_smelting_min_max_df["Difference"] = (
    mining_smelting_min_max_df["Max Total"] - mining_smelting_min_max_df["Min Total"]
)
mining_smelting_min_max_df["More Expensive"] = (
    mining_smelting_min_max_df["Max Total"]
    / mining_smelting_min_max_df["Min Total"]
    * 100
    - 100
)

# Format the total columns to two decimal places
mining_smelting_min_max_df["Min Total"] = mining_smelting_min_max_df["Min Total"].map(
    lambda x: f"{x:.2f}"
)
mining_smelting_min_max_df["Max Total"] = mining_smelting_min_max_df["Max Total"].map(
    lambda x: f"{x:.2f}"
)
mining_smelting_min_max_df["Difference"] = mining_smelting_min_max_df["Difference"].map(
    lambda x: f"{x:.2f}"
)
mining_smelting_min_max_df["More Expensive"] = mining_smelting_min_max_df[
    "More Expensive"
].map(lambda x: f"{x:.2f}%")

# Display the DataFrame as a table
display(mining_smelting_min_max_df)

# Plot the lines for each ore's min, max, and combined min and max values
fig, ax = plt.subplots()

ores = mining_smelting_min_max_df["Ore"]
mining_min_costs = mining_smelting_min_max_df["Mining Min Cost"].astype(float)
smelting_min_costs = mining_smelting_min_max_df["Smelting Min Cost"].astype(float)
min_totals = mining_smelting_min_max_df["Min Total"].astype(float)
mining_max_costs = mining_smelting_min_max_df["Mining Max Cost"].astype(float)
smelting_max_costs = mining_smelting_min_max_df["Smelting Max Cost"].astype(float)
max_totals = mining_smelting_min_max_df["Max Total"].astype(float)

ax.plot(ores, mining_min_costs, label="Mining Min", marker="o")
ax.plot(ores, smelting_min_costs, label="Smelting Min", marker="o")
ax.plot(ores, min_totals, label="Min Total", marker="o")
ax.plot(ores, mining_max_costs, label="Mining Max", marker="o")
ax.plot(ores, smelting_max_costs, label="Smelting Max", marker="o")
ax.plot(ores, max_totals, label="Max Total", marker="o")

ax.axhline(y=100, color="red", linestyle="--", label="100")

ax.set_xlabel("Ore")
ax.set_ylabel("Cost (gold)")
ax.set_title("Cost per 100 units of ores (Min, Max, and Combined)")
ax.legend()
ax.grid(True)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

if ___SAVE_IMAGES__:
    save_plot_as_file(plt, "mining_smelting_min_max_cost_per_100_units")

Rank 11 Mining grants Gold equal to the amount of Ore, which means that every ore besides verisium is self sustaining, while it's close with 99.6 for bismuth in the worst case. When using workers with a slightly above average wage it might be not.
The less efficient workers cost up to ~35% more gold. It should be noted that mining as smelting are the cheapest skills, while having a high uptime compared to other skills.

In [ ]:
# Farming: calculate the wage for 100 producted crops
farming_wage = wage_dict[values.FARMING]
farming_output = worker_output_dict[values.FARMING]

# Dictionary to store the cost per 100 units for each ore
farming_cost_per_100_units = {crop: [] for crop in values.CROPS}

for crop in values.CROPS:
    print(f"Farming {crop}:")
    for rank in range(1, 11):
        wage = farming_wage[rank]
        output = farming_output[crop][rank]
        if not output:
            farming_cost_per_100_units[crop].append(None)
            continue
        cost = wage / output * 100
        farming_cost_per_100_units[crop].append(cost)
        if __DEBUG__:
            print(f"Rank {rank}: {cost:.2f} gold per 100 {crop}")
    print()

# Create a DataFrame from the smelting costs
smelting_df = pd.DataFrame(farming_cost_per_100_units, index=range(1, 11))

# Plot the cost per 100 units for each ore
fig, ax = plt.subplots()

for crop, costs in farming_cost_per_100_units.items():
    ranks = range(1, 11)
    ax.plot(ranks, costs, label=crop, marker="o")

ax.set_xticks(range(1, 11))
ax.set_xlabel("Rank")
ax.set_ylabel("Cost per 100 units (gold)")
ax.set_title("Cost per 100 units of farmed crops")
ax.legend()
ax.grid(True)

plt.show()

if ___SAVE_IMAGES__:
    save_plot_as_file(plt, "farming_cost_per_100_units")